In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from scipy import stats
from statsmodels.tsa.api import VAR
from statsmodels.tools.eval_measures import rmse, aic
from statsmodels.tsa.ar_model import AutoReg
import pickle

In [75]:
df_aapl = pd.read_csv("df_aaple.csv")
df_small = df_aapl.iloc[:,:4]
df_small.drop(columns="Adj. Close", inplace=True)
df_small["P/E"] = df_small["P/E (LTM)"]
df_small.drop(columns="P/E (LTM)", inplace=True)
df_small["# Buys"] = df_aapl["# Buys"]

df_small_raw = df_small

In [67]:
df_aapl

,Date,Close,Adj. Close,P/E (LTM),EPS - Est High (NTM),EPS - Est Low (NTM),Volume,SI (%),Vol,# Buys,...,day_6,day_7,day_8,day_9,day_10,day_11,day_12,day_13,day_14,day_15
0,1/9/2003,27.905,17.580627,35.994469,1.0800,0.900,62048642.0,0.953692,0.307837,14.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1/10/2003,27.960,17.615278,36.065413,1.0800,0.900,67985474.0,0.953692,0.306803,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1/13/2003,28.195,17.763332,36.368538,1.0800,0.900,61040600.0,0.953692,0.305531,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1/14/2003,28.485,17.946037,36.742607,1.0800,0.915,54441838.0,0.953692,0.304476,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1/15/2003,28.135,17.725531,36.291144,1.0800,0.915,59999384.0,0.953692,0.284455,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4249,11/25/2019,151.230,148.094575,28.533935,5.9759,5.120,22428590.0,0.810456,0.122422,23.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
4250,11/26/2019,152.030,148.877988,28.684879,5.9759,5.120,24635100.0,0.821984,0.096989,23.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
4251,11/27/2019,152.320,149.161976,28.739596,5.9759,5.120,15201290.0,0.821984,0.086451,23.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
4252,11/29/2019,151.380,148.241465,28.562237,5.9759,5.120,11977300.0,0.821984,0.085595,23.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0


In [76]:

features = list(df_small.columns)[1:]


for feature in features:
    result = adfuller(df_small[feature], autolag=None)
    counter = 0
    while result[1] > 0.05:
        df_small[feature] = df_small[feature] - df_small[feature].shift(1)
        #df_small.dropna()
        counter += 1
        #dropna(inplace=False) because it drops one observation for each feature
        result = adfuller(df_small.dropna()[feature], autolag=None)
    print(f'Order of integration for feature "{feature}" is {counter}')
df_small.dropna(inplace=True)
df_small.reset_index(drop=True, inplace=True)
        

Order of integration for feature "Close" is 1
Order of integration for feature "P/E" is 1
Order of integration for feature "# Buys" is 1


In [78]:
BICs = []
for i in list(range(40)):
    model = AutoReg(df_small.iloc[:,1], lags=i).fit()
    BICs.append(model.bic)

min_bic_ind = BICs.index(min(BICs))

model = AutoReg(df_small.iloc[:,1], lags=min_bic_ind).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            AutoReg Model Results                             
==============================================================================
Dep. Variable:                  Close   No. Observations:                 4253
Model:                     AutoReg(8)   Log Likelihood               -4794.674
Method:               Conditional MLE   S.D. of innovations              0.749
Date:                Wed, 11 May 2022   AIC                           9609.348
Time:                        22:45:05   BIC                           9672.883
Sample:                             8   HQIC                          9631.801
                                 4253                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0405      0.012      3.495      0.000       0.018       0.063
Close.L1      -0.0993      0.015     -6.485      0.000      -0.129      -0.069
Close.L2      -0.0826      0.015     -5.369      0.000      -0.113      -0.052
Close.L3       0.0275      0.015      1.785      0.074      -0.003       0.058
Close.L4      -0.0572      0.015     -3.721      0.000      -0.087      -0.027
Close.L5      -0.0704      0.015     -4.571      0.000      -0.101      -0.040
Close.L6      -0.0306      0.015     -1.984      0.047      -0.061      -0.000
Close.L7       0.0095      0.015      0.619      0.536      -0.021       0.040
Close.L8      -0.0834      0.015     -5.449      0.000      -0.113      -0.053
                                    Roots                                    
=============================================================================
                  Real          Imaginary           Modulus         Frequency
-----------------------------------------------------------------------------
AR.1           -1.2366           -0.5450j            1.3513           -0.4339
AR.2           -1.2366           +0.5450j            1.3513            0.4339
AR.3           -0.4849           -1.1955j            1.2901           -0.3113
AR.4           -0.4849           +1.1955j            1.2901            0.3113
AR.5            1.2130           -0.6232j            1.3637           -0.0755
AR.6            1.2130           +0.6232j            1.3637            0.0755
AR.7            0.5656           -1.3421j            1.4564           -0.1865
AR.8            0.5656           +1.3421j            1.4564            0.1865
-----------------------------------------------------------------------------
"""

In [93]:
len(columns)

39

In [94]:
columns = []
for i in list(range(1,40)):
    columns.append(features[1]+".L"+str(i))

feature_n_df = pd.DataFrame(columns=columns)
for i in list(range(39)):
    feature_n_df[columns[i]] = df_small[features[1]].shift(i)

feature_n_df.fillna(1, inplace=True)

BICs = []
for i in list(range(39)):
    model = AutoReg(df_small.iloc[:,1], lags=min_bic_ind, exog=feature_n_df.iloc[:,:i+1]).fit()
    BICs.append(model.bic)

min_bic_ind_aug = BICs.index(min(BICs))


model = AutoReg(df_small.iloc[:,1], lags=min_bic_ind, exog=feature_n_df.iloc[:,:min_bic_ind_aug+1]).fit()

#columns
model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            AutoReg Model Results                             
==============================================================================
Dep. Variable:                  Close   No. Observations:                 4253
Model:                   AutoReg-X(8)   Log Likelihood               -4293.157
Method:               Conditional MLE   S.D. of innovations              0.665
Date:                Wed, 11 May 2022   AIC                           8614.314
Time:                        23:02:30   BIC                           8703.263
Sample:                             8   HQIC                          8645.749
                                 4253                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0376      0.010      3.649      0.000       0.017       0.058
Close.L1      -0.0688      0.015     -4.498      0.000      -0.099      -0.039
Close.L2      -0.0807      0.015     -5.268      0.000      -0.111      -0.051
Close.L3       0.0524      0.015      3.410      0.001       0.022       0.082
Close.L4      -0.0549      0.014     -4.015      0.000      -0.082      -0.028
Close.L5      -0.0479      0.014     -3.498      0.000      -0.075      -0.021
Close.L6      -0.0278      0.014     -2.025      0.043      -0.055      -0.001
Close.L7       0.0164      0.014      1.197      0.231      -0.010       0.043
Close.L8      -0.0605      0.014     -4.441      0.000      -0.087      -0.034
P/E.L1         0.4255      0.013     32.928      0.000       0.400       0.451
P/E.L2        -0.0355      0.014     -2.454      0.014      -0.064      -0.007
P/E.L3         0.0026      0.014      0.178      0.858      -0.026       0.031
P/E.L4        -0.0700      0.014     -4.833      0.000      -0.098      -0.042
                                    Roots                                    
=============================================================================
                  Real          Imaginary           Modulus         Frequency
-----------------------------------------------------------------------------
AR.1           -1.2627           -0.5953j            1.3960           -0.4299
AR.2           -1.2627           +0.5953j            1.3960            0.4299
AR.3           -0.4911           -1.2416j            1.3352           -0.3100
AR.4           -0.4911           +1.2416j            1.3352            0.3100
AR.5            1.2768           -0.6376j            1.4272           -0.0737
AR.6            1.2768           +0.6376j            1.4272            0.0737
AR.7            0.6122           -1.4002j            1.5282           -0.1844
AR.8            0.6122           +1.4002j            1.5282            0.1844
-----------------------------------------------------------------------------
"""

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from scipy import stats
from statsmodels.tsa.api import VAR
from statsmodels.tools.eval_measures import rmse, aic
from statsmodels.tsa.ar_model import AutoReg
import pickle

df_aapl = pd.read_csv("df_aaple.csv")
df_small = df_aapl.iloc[:,:4]
df_small.drop(columns="Adj. Close", inplace=True)
df_small["P/E"] = df_small["P/E (LTM)"]
df_small.drop(columns="P/E (LTM)", inplace=True)
df_small["# Buys"] = df_aapl["# Buys"]

df_small_raw = df_small

def algo(df, max_lag):

    # Step 1: Tranformation for stationarity
    features = list(df_small.columns)[1:]

    for feature in features:
        result = adfuller(df_small[feature], autolag=None)
        counter = 0
        while result[1] > 0.05:
            df_small[feature] = df_small[feature] - df_small[feature].shift(1)
            #df_small.dropna()
            counter += 1
            #dropna(inplace=False) because it drops one observation for each feature
            result = adfuller(df_small.dropna()[feature], autolag=None)
        print(f'Order of integration for feature "{feature}" is {counter}')
    df_small.dropna(inplace=True)
    df_small.reset_index(drop=True, inplace=True)

    # Step 2: Building a univariate model and finding the optimal l
    BICs = []
    for i in list(range(max_lag)):
        model = AutoReg(df_small.iloc[:,1], lags=i).fit()
        BICs.append(model.bic)

    min_bic_ind = BICs.index(min(BICs))

    # model = AutoReg(df_small.iloc[:,1], lags=min_bic_ind).fit()
    # model.summary()


    # Step 2: Bulding augmented model and finding the optimal w for each Xi
    
    # Defining dictionary to store all augmented models
    aug_models = {}
    feature_n_dfs = {}

    
    for n in list(range(1, len(features))):
        columns = []
        for i in list(range(1, max_lag)):
            columns.append(features[n]+".L"+str(i))

        feature_n_df = pd.DataFrame(columns=columns)
        for i in list(range(max_lag - 1)):
            feature_n_df[columns[i]] = df_small[features[n]].shift(i)

        feature_n_df.fillna(1, inplace=True)

        BICs = []
        for i in list(range(max_lag - 1)):
            model = AutoReg(df_small.iloc[:,1], lags=min_bic_ind, exog=feature_n_df.iloc[:,:i+1]).fit()
            BICs.append(model.bic)

        min_bic_ind_aug = BICs.index(min(BICs))

        model = AutoReg(df_small.iloc[:,1], lags=min_bic_ind, exog=feature_n_df.iloc[:,:min_bic_ind_aug+1]).fit()

        aug_models[features[n]] = model
        feature_n_dfs[features[n]] = feature_n_df
        #model.summary()
    return aug_models, feature_n_dfs



In [119]:
list(range(1, len(features)))

[1, 2]

In [4]:
aug_models, dfs = algo(df=df_small, max_lag=40)

Order of integration for feature "Close" is 1
Order of integration for feature "P/E" is 1
Order of integration for feature "# Buys" is 1


In [134]:
dfs['# Buys']

,# Buys.L1,# Buys.L2,# Buys.L3,# Buys.L4,# Buys.L5,# Buys.L6,# Buys.L7,# Buys.L8,# Buys.L9,# Buys.L10,...,# Buys.L30,# Buys.L31,# Buys.L32,# Buys.L33,# Buys.L34,# Buys.L35,# Buys.L36,# Buys.L37,# Buys.L38,# Buys.L39
0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,0.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,0.0,0.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,0.0,0.0,0.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,0.0,0.0,0.0,0.0,-1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4248,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4249,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4251,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [5]:
aug_models['# Buys'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            AutoReg Model Results                             
==============================================================================
Dep. Variable:                  Close   No. Observations:                 4253
Model:                   AutoReg-X(8)   Log Likelihood               -4785.470
Method:               Conditional MLE   S.D. of innovations              0.747
Date:                Fri, 13 May 2022   AIC                           9592.940
Time:                        14:28:54   BIC                           9662.828
Sample:                             8   HQIC                          9617.638
                                 4253                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0401      0.012      3.465      0.001       0.017       0.063
Close.L1      -0.0985      0.015     -6.448      0.000      -0.128      -0.069
Close.L2      -0.0834      0.015     -5.430      0.000      -0.113      -0.053
Close.L3       0.0278      0.015      1.805      0.071      -0.002       0.058
Close.L4      -0.0577      0.015     -3.760      0.000      -0.088      -0.028
Close.L5      -0.0712      0.015     -4.635      0.000      -0.101      -0.041
Close.L6      -0.0306      0.015     -1.991      0.046      -0.061      -0.000
Close.L7       0.0088      0.015      0.570      0.568      -0.021       0.039
Close.L8      -0.0836      0.015     -5.475      0.000      -0.114      -0.054
# Buys.L1      0.2098      0.049      4.295      0.000       0.114       0.306
                                    Roots                                    
=============================================================================
                  Real          Imaginary           Modulus         Frequency
-----------------------------------------------------------------------------
AR.1           -1.2378           -0.5451j            1.3526           -0.4340
AR.2           -1.2378           +0.5451j            1.3526            0.4340
AR.3           -0.4853           -1.1947j            1.2895           -0.3114
AR.4           -0.4853           +1.1947j            1.2895            0.3114
AR.5            1.2110           -0.6231j            1.3619           -0.0756
AR.6            1.2110           +0.6231j            1.3619            0.0756
AR.7            0.5645           -1.3420j            1.4559           -0.1866
AR.8            0.5645           +1.3420j            1.4559            0.1866
-----------------------------------------------------------------------------
"""

In [149]:
model = VAR(df_small_raw.iloc[:,1:3])
for i in [1,2]:
    result = model.fit(i)
    try:
        print('Lag Order =', i)
        print('BIC:', result.bic)
    except:
        continue

result.params['Close']
result.summary()


Lag Order = 1
BIC: -1.2397748793967445
Lag Order = 2
BIC: -1.2393283679890952


c:\Users\Anton\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Wed, 11, May, 2022
Time:                     17:19:19
--------------------------------------------------------------------
No. of Equations:         2.00000    BIC:                   -1.23933
Nobs:                     4250.00    HQIC:                  -1.24900
Log likelihood:          -9385.63    FPE:                   0.285281
AIC:                     -1.25428    Det(Omega_mle):        0.284611
--------------------------------------------------------------------
Results for equation Close
              coefficient       std. error           t-stat            prob
---------------------------------------------------------------------------
const            0.033028         0.011606            2.846           0.004
L1.Close        -0.079371         0.017203           -4.614           0.000
L1.P/E          -0.045465         0.016396           -2.773           0.006
L2.C

In [138]:
result.2

SyntaxError: invalid syntax (1700330168.py, line 1)

In [104]:
result = adfuller(df_small["Close"])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

result = adfuller(df_small["P/E"])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))



ADF Statistic: 4.937903
p-value: 1.000000
Critical Values:
	1%: -3.432
	5%: -2.862
	10%: -2.567
ADF Statistic: -2.421544
p-value: 0.135748
Critical Values:
	1%: -3.432
	5%: -2.862
	10%: -2.567


In [73]:
df_small['Close'] = df_small['Close'] - df_small['Close'].shift(1)
df_small['P/E'] = df_small['P/E'] - df_small['P/E'].shift(1)


df_small.dropna(inplace=True)

In [66]:
grangercausalitytests(df_small[["Close","P/E"]], maxlag=[5])


Granger Causality
number of lags (no zero) 5
ssr based F test:         F=7.7118  , p=0.0000  , df_denom=4237, df_num=5
ssr based chi2 test:   chi2=38.6589 , p=0.0000  , df=5
likelihood ratio test: chi2=38.4840 , p=0.0000  , df=5
parameter F test:         F=7.7118  , p=0.0000  , df_denom=4237, df_num=5


{5: ({'ssr_ftest': (7.711754967300268, 3.1386317913070684e-07, 4237.0, 5),
   'ssr_chi2test': (38.65888022314319, 2.781446155988556e-07, 5),
   'lrtest': (38.484032849817595, 3.0160195097801265e-07, 5),
   'params_ftest': (7.711754967300332, 3.1386317913070684e-07, 4237.0, 5.0)},
   array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])])}

In [67]:
grangercausalitytests(df_small[["P/E", "Close"]], maxlag=[5])


Granger Causality
number of lags (no zero) 5
ssr based F test:         F=2.8986  , p=0.0129  , df_denom=4237, df_num=5
ssr based chi2 test:   chi2=14.5305 , p=0.0126  , df=5
likelihood ratio test: chi2=14.5057 , p=0.0127  , df=5
parameter F test:         F=2.8986  , p=0.0129  , df_denom=4237, df_num=5


{5: ({'ssr_ftest': (2.898580563397515, 0.012866690671738696, 4237.0, 5),
   'ssr_chi2test': (14.530528951277606, 0.012568454320955129, 5),
   'lrtest': (14.505734220321756, 0.01269681847337552, 5),
   'params_ftest': (2.8985805633976964, 0.012866690671735884, 4237.0, 5.0)},
   array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])])}

,Date,Close,P/E (LTM)
0,1/9/2003,27.905,35.994469
1,1/10/2003,27.960,36.065413
2,1/13/2003,28.195,36.368538
3,1/14/2003,28.485,36.742607
4,1/15/2003,28.135,36.291144
...,...,...,...
4249,11/25/2019,151.230,28.533935
4250,11/26/2019,152.030,28.684879
4251,11/27/2019,152.320,28.739596
4252,11/29/2019,151.380,28.562237
